In [1]:
import os
import argparse
import json
import copy
import torch
import numpy as np

from pykt.models import evaluate,evaluate_question,load_model
from pykt.datasets import init_test_datasets

device = "cpu" if not torch.cuda.is_available() else "cuda"
os.environ['CUBLAS_WORKSPACE_CONFIG']=':4096:2'

with open("../configs/wandb.json") as fin:
    wandb_config = json.load(fin)

def main(params):
    if params['use_wandb'] ==1:
        import wandb
        os.environ['WANDB_API_KEY'] = wandb_config["api_key"]
        wandb.init(project="wandb_predict")

    save_dir, batch_size, fusion_type = params["save_dir"], params["bz"], params["fusion_type"].split(",")
    win200 = params["win200"]

    with open(os.path.join(save_dir, "config.json")) as fin:
        config = json.load(fin)
        model_config = copy.deepcopy(config["model_config"])
        for remove_item in ['use_wandb','learning_rate','add_uuid','l2']:
            if remove_item in model_config:
                del model_config[remove_item]    
        trained_params = config["params"]
        model_name, dataset_name, emb_type, fold = trained_params["model_name"], trained_params["dataset_name"], trained_params["emb_type"], trained_params["fold"]
        if model_name in ["saint", "sakt", "cdkt"]:
            train_config = config["train_config"]
            seq_len = train_config["seq_len"]
            model_config["seq_len"] = seq_len   
        if model_name in ["stosakt"]:
            train_args = argparse.ArgumentParser()
            args_dict = vars(train_args)
            args_dict.update(config["train_args"])
            # print(f"train_args:{train_args.hidden_size}")

    with open("../configs/data_config.json") as fin:
        curconfig = copy.deepcopy(json.load(fin))
        if model_name in ["gpt4kt"]:
            dataset_name = params["dataset_name"]
        data_config = curconfig[dataset_name]
        data_config["dataset_name"] = dataset_name
        if model_name in ["dkt_forget", "bakt_time"] or emb_type.find("time") != -1:
            data_config["num_rgap"] = config["data_config"]["num_rgap"]
            data_config["num_sgap"] = config["data_config"]["num_sgap"]
            data_config["num_pcount"] = config["data_config"]["num_pcount"]
        elif model_name in ["lpkt"]:
            print("running  prediction")
            data_config["num_at"] = config["data_config"]["num_at"]
            data_config["num_it"] = config["data_config"]["num_it"] 
        elif model_name in ["gpt4kt"]:
            data_config["num_q"] = config["data_config"]["num_q"]
            data_config["num_c"] = config["data_config"]["num_c"] 
            
    # test_loader, test_window_loader, test_question_loader, test_question_window_loader = init_test_datasets(data_config, model_name, batch_size,fold,win200)

    print(f"Start predicting model: {model_name}, embtype: {emb_type}, save_dir: {save_dir}, dataset_name: {dataset_name}")
    print(f"model_config: {model_config}")
    print(f"data_config: {data_config}")

    if model_name in ["stosakt"]:
        model = load_model(model_name, model_config, data_config, emb_type, save_dir, train_args)
        print(f"model_parameter:{sum(dict((p.data_ptr(), p.numel()) for p in model.parameters() if p.requires_grad).values())}")
    else:
        model = load_model(model_name, model_config, data_config, emb_type, save_dir)
        print(f"model_parameter:{sum(dict((p.data_ptr(), p.numel()) for p in model.parameters() if p.requires_grad).values())}")

In [54]:
params={
'bz': 512,
"fusion_type":'late_fusion',
'use_wandb':0,
'dataset_name':'nips_task34',
'win200':True,
'save_dir':"/mnt/cfs/project2/full_result_pykt/best_model_path/ednet/sparsekt-topk/sparsekt_tiaocan_ednet_3407_0_0.3_64_256_2_4_0.5_0.5_0.5_50_256_256_8_4_0.0001_0.9_4_1_1_1_98714f5d-3e31-4baf-b9f7-28ce256b7b4f"
}

In [55]:
main(params)

Start predicting model: sparsekt, embtype: qid_sparseattn, save_dir: /mnt/cfs/project2/full_result_pykt/best_model_path/ednet/sparsekt-topk/sparsekt_tiaocan_ednet_3407_0_0.3_64_256_2_4_0.5_0.5_0.5_50_256_256_8_4_0.0001_0.9_4_1_1_1_98714f5d-3e31-4baf-b9f7-28ce256b7b4f, dataset_name: ednet
model_config: {'dropout': 0.3, 'final_fc_dim': 64, 'final_fc_dim2': 256, 'num_layers': 2, 'nheads': 4, 'loss1': 0.5, 'loss2': 0.5, 'loss3': 0.5, 'start': 50, 'd_model': 256, 'd_ff': 256, 'num_attn_heads': 8, 'n_blocks': 4, 'sparse_ratio': 0.9, 'k_index': 4, 'stride': 1}
data_config: {'dpath': '../data/ednet', 'num_q': 11901, 'num_c': 188, 'input_type': ['questions', 'concepts'], 'max_concepts': 7, 'min_seq_len': 3, 'maxlen': 200, 'emb_path': '', 'train_valid_original_file': 'train_valid.csv', 'train_valid_file': 'train_valid_sequences.csv', 'folds': [0, 1, 2, 3, 4], 'test_original_file': 'test.csv', 'test_file': 'test_sequences.csv', 'test_window_file': 'test_window_sequences.csv', 'test_question_file'